In [10]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('Finalcontent.db')

# Fetch URLs and content from the documents table
df = pd.read_sql_query("SELECT url, page_content FROM documents", conn)

# Close the connection
conn.close()

# Display the fetched data (optional)
df.head()


,url,page_content
0,https://thecurrent.pk/thecurrent-blogs/,\n\n\n\n THECURRENT BLOGS - The Current Pk\n\n...
1,https://thecurrent.pk/shoaib-bashir-about-muha...,\n\n\n\n England's Shoaib Bashir reveals Muham...
2,https://thecurrent.pk/power-play-hope-for-busi...,\n\n\n\n Power play: Hope for businesses with ...
3,https://thecurrent.pk/asma-abbas/,\n\n\n\n 'My husband didn’t allow me to work i...
4,https://thecurrent.pk/screen-legend-shafqat-ch...,\n\n\n\n Screen legend Shafqat Cheema is in a ...


In [12]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained embedding model
model = SentenceTransformer('all-mpnet-base-v2')

# Generate embeddings for each content
embeddings = model.encode(df['page_content'].tolist(), show_progress_bar=True)

# Add the embeddings to the dataframe
df['embedding'] = embeddings.tolist()


d:\Data Science Journey\theMultyTeam\env\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Abdullah\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Batches: 100%|██████████| 32/32 [03:33<00:00,  6.68s/it]


In [16]:
import chromadb

# Initialize ChromaDB client
client = chromadb.PersistentClient(path="./chroma_vector_dbfinal")
collection = client.create_collection("documents_collection")

# Add data to the collection
collection.add(
    ids=[str(i) for i in df.index],  # Unique ids (use document index as id)
    documents=df['page_content'].tolist(),  # Document content
    embeddings=df['embedding'].tolist(),  # Corresponding vector embeddings
    metadatas=[{"url": url} for url in df['url']]  # Metadata with URLs
)


In [45]:
# Step 5: Query Function to Search the Vector Database
def query_vector_db(user_query, top_k=5):
    query_embedding = model.encode([user_query])  # Encode the user query

    # Perform similarity search
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k
    )
    context = ""

    for doc,url in zip(results["documents"][0],results['metadatas'][0]):

        # doc = doc.replace("\n\n"," ")
        context += f"URL: {url['url']} \n\n BlogPost:\n {doc.replace("\n\n"," ")}"
    # # Display results (URLs and content)
    # for i, metadata in enumerate(results['metadatas']):
    #     print(f"Result {i+1}:")
    #     # print(f"URL: {metadata['url']}")
    #     print(f"Page Content: {results['documents'][i][0]}")
    #     print()
    return context
# Example usage
output = query_vector_db("Zakir Naik Visik to Pakistan", top_k=3)

In [39]:
print(output['metadatas'][0])

[{'url': 'https://thecurrent.pk/dr-zakir-naik-what-a-problematic-man-and-what-a-controversial-tour/'}, {'url': 'https://thecurrent.pk/celebrities-journalists-slam-zakir-naik-for-misbehaving-during-pakistan-tour/'}, {'url': 'https://thecurrent.pk/dr-zakir-naik-faces-criticism-for-complaining-pias-overweight-charges/'}]


In [43]:
len(output['documents'][0])

3

In [42]:
print(output['documents'][0][1].replace("\n\n"," ").replace("\t\t"," "))

   Celebrities, journalists slam Zakir Naik for misbehaving during Pakistan tour - The Current Pk         
                                                                
      
          
Search 
                                        
Search 
              
                                           Celebrities, journalists slam Zakir Naik for misbehaving during Pakistan tour           
                
Celebrities and journalists have slammed Indian scholar Dr Zakir Naik for behaving disrespectfully and rudely in Pakistan. His insulting remarks about women have left many people upset.
Controversial remarks on hospitality:                
Speaking at the Governor House in Karachi, he was offended by recalling his tour to Pakistan and stated that the PIA Chief Executive Officer (CEO) offered him a 50 per cent discount on his overloaded 500 KG baggage and questioned Pakistan:s hospitality.
Zakir Naik said, “If I were in an Indian airport, they would even let me load 1000 KG of baggag